In [ ]:
% matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot as plt
from matplotlib import cm
from matplotlib import animation
from drift_qec.base import UnitalChannel
import numpy as np
import scipy as sp
import seaborn as sns
sns.set_style("whitegrid")

In [ ]:
def Qx(w):
    return np.array([[1,          0,         0],
                     [0,  np.cos(w), np.sin(w)],
                     [0, -np.sin(w), np.cos(w)]])

def Qz(w):
    return np.array([[ np.cos(w), np.sin(w), 0],
                     [-np.sin(w), np.cos(w), 0],
                     [         0,         0, 1]])

def Q(w1, w2, w3):
    return np.dot(np.dot(Qz(w1), Qx(w2)), Qz(w3))

def R(a1, a2, a3):
    return np.diag(np.array([a1, a2, a3]))

In [ ]:
max_time = 100
ax, ay, az = [], [], []
px, py, pz = [], [], []

channel = UnitalChannel(0.1)
for time in range(max_time):
    ax.append(channel.A[0, 0])
    ay.append(channel.A[1, 1])
    az.append(channel.A[2, 2])
    px.append(channel.PROB[0, 0])
    py.append(channel.PROB[1, 1])
    pz.append(channel.PROB[2, 2])
    channel.step()

plt.plot(ax, label="ax")
plt.plot(ay, label="ay")
plt.plot(az, label="az")
plt.plot(px, label="px", ls="--")
plt.plot(py, label="py", ls="--")
plt.plot(pz, label="pz", ls="--")
plt.legend(frameon=True)

In [ ]:
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [0, 1j]])
Z = np.array([[1, 0], [0, -1]])

In [ ]:
channel.step()
u, v = np.mgrid[0:2*np.pi:20j, 0:np.pi:20j]
x, y, z = np.cos(u)*np.sin(v), np.sin(u)*np.sin(v), np.cos(v)

fig = plt.figure(figsize=(14,10))
ax = plt.axes(projection='3d')
ax.set_aspect("equal")
ax.set_frame_on(False)
ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.w_yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.w_zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.grid(False)

ax.plot_wireframe(x, y, z, color=[0.9, 0.9, 0.9], linewidth=0.75)
ax.plot_surface(*channel.get_verts(x, y, z), rstride=1, cstride=1, cmap=cm.coolwarm)

# Euler angle estimation

In [ ]:
Q(0.1, 0.0, -0.1)

In [ ]:
np.dot(Q(0.1, 0.2, 0.3), R(0.99, 0.8, 0.7))

# Drift rates vs granularity
Two grid cells are equivalent, and therefore should be joined (i.e.: the granularity should be decreased), if their probabilities are similar. For the normal distribution, this should be calculated for the grid cells that neighbour the greatest increase/decrease in the pdf. Inflection points on a normal curve are at $\pm \sigma$. At an inflection point, the absolute value of the first derivative is

$$1 / (\sigma^2 \sqrt{2 \pi e})$$

Thus, if we want the probability $p$ to remain within $\epsilon$ of the original value, we must have grids that are not more than $\epsilon \sigma^2 \sqrt{2 \pi e}$ apart. In other words, the number of grid points scales as $1/\sigma^2$. But $\sigma^2$ itself only scales as $\kappa^2 N$. So we have to use $\kappa^2$.

In [1]:
import numpy as np

In [7]:
def Qx(w):
    d = w.shape
    return np.array([[ np.ones(d), np.zeros(d), np.zeros(d)],
                     [np.zeros(d),   np.cos(w),   np.sin(w)],
                     [np.zeros(d),  -np.sin(w),   np.cos(w)]])

def Qz(w):
    d = w.shape
    return np.array([[   np.cos(w),   np.sin(w), np.zeros(d)],
                     [  -np.sin(w),   np.cos(w), np.zeros(d)],
                     [ np.zeros(d), np.zeros(d),  np.ones(d)]])

def getQ(grains=100):
    theta1 = np.linspace(0, np.pi, grains)
    psi = np.linspace(0, 2*np.pi, grains)
    theta2 = np.linspace(0, np.pi, grains)
    Q1 = Qz(theta1)
    Q2 = Qx(psi)
    Q3 = Qz(theta2)
    Q12 = np.tensordot(Q1, Q2, axes=([0], [1]))
    Q12 = np.swapaxes(Q12, 1, 2)
    Q123 = np.tensordot(Q12, Q3, axes=([0], [1]))
    Q123 = np.swapaxes(np.swapaxes(Q123, 1, 3), 2, 3)
    return Q123

def Q_probs(Q, kappa=0.1):
    pass



In [12]:
kappa = 0.1
Q = getQ()
R = np.diag([1, kappa, 0])